In [1]:
import random
import sys
from time import time
from collections import deque, defaultdict, namedtuple
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import csv
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
import mip

# Define device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
device

device(type='cpu')

In [3]:
# ================================================================
#  nmg30_scenario_evaluator.py
# ================================================================

from __future__ import annotations
from typing import Dict, Sequence, Tuple
import numpy as np
from mip import Model, xsum, BINARY, MAXIMIZE, OptimizationStatus

# ----------------------------------------------------------------
# 1. GLOBAL CONFIGURATION
# ----------------------------------------------------------------
T        = 24          # hours
Delta_t  = 1.0         # h
M        = 1e3         # Big-M

# ---------------- 1.1  Network size -----------------------------
NUM_MGC  = 9                     # controllers: internal idx 0‒8
NUM_MG   = 21                    # micro-grids:  idx 0‒20
MGC      = list(range(NUM_MGC))  # convenience

# ---------------- 1.2  Physical-bus mapping ---------------------
#   (internal index → physical bus number)
PV_MG_BUS   = {0: 4,  1: 5,  2: 7,  3: 8,  4: 9,  5: 11}
MGC_BUS     = {0: 1,  1: 2,  2: 3,  3: 6,  4: 13,
               5: 22, 6: 23, 7: 27, 8: 28}
WIND_MG_BUS = {6: 10, 7: 12, 8: 14, 9: 15, 10: 16,
               11: 17, 12: 18, 13: 19, 14: 20, 15: 21,
               16: 24, 17: 25, 18: 26, 19: 29, 20: 30}

# Union of all MG indices (0‒20)
MG_BUS = {**PV_MG_BUS, **WIND_MG_BUS}

# ---------------- 1.3  Derived index sets -----------------------
PV_MGs    = sorted(PV_MG_BUS.keys())        # [0,1,2,3,4,5]
WIND_MGs  = sorted(WIND_MG_BUS.keys())      # [6,7,…,20]
BATT_MG   = PV_MGs                          # if batteries only on PV MGs
WIND_MGCs = MGC                             # every controller has wind

# ---------------------------------------------------------------
# 5. CORE SYSTEM PARAMETERS  – 30-Bus version
# ---------------------------------------------------------------

# 5.1  Microgrid-Controller fleet  (nine controllers)
P_MT_MGC_MIN  = [0] * 9                   # MW   (all start from 0)
P_MT_MGC_MAX  = [6.0, 6.0, 5.5, 5.0, 6.5,
                 5.5, 5.5, 5.0, 5.0]      # edit as needed
RU_MGC        = [3.0, 3.0, 2.8, 2.5, 3.2,
                 3.0, 3.0, 2.6, 2.6]       # MW / h
RD_MGC        = RU_MGC[:]                 # symmetric ramps for now
MIN_UP        = [2] * 9                   # h   minimum up-time
MIN_DOWN      = [2] * 9
STARTUP_COST  = [2.2, 2.3, 2.1, 2.0, 2.4,
                 2.2, 2.3, 2.1, 2.0]      # k$
SHUTDOWN_COST = [0.7, 0.6, 0.6, 0.6, 0.7,
                 0.7, 0.6, 0.6, 0.6]
ETA_MGC       = [0.42, 0.43, 0.41, 0.40, 0.44,
                 0.42, 0.42, 0.41, 0.40]   # electrical efficiency
C_NG_MGC      = [15.0, 14.8, 14.5, 14.5, 15.2,
                 14.9, 15.0, 14.7, 14.7]   # $/MMBtu
L_NG_MGC      = [3] * 9                   # tCO₂/MWh (placeholder)

P_GRID_MAX    = [20] * 9                 # MW import cap (raise later if needed)

MARKUP        = [1.10] * 9               # 10 % retail margin over grid
C_GRID        = [  # sample 24-h price vector
    50.61, 49.88, 49.41, 47.86, 46.07, 45.07, 45.68, 47.44,
    49.44, 49.78, 46.09, 45.60, 45.04, 43.54, 35.85, 43.52,
    44.41, 49.00, 52.22, 52.39, 52.91, 53.42, 52.97, 51.51
]
C_EXC = [[round(p * MARKUP[i], 2) for p in C_GRID] for i in range(9)]

P_MGC_MAX   = 35                       # MW : Σ exports cap per controller
LAMBDA_ATC  = 0.5                      # redispatch penalty coeff.


# ---------------------------------------------------------------
# 5.2  Local-MG micro-turbine data
# ---------------------------------------------------------------
NUM_MG  = 21                           # MG-0 … MG-20
C_NG    = 4.0                          # $/MMBtu  (uniform for MG MTs)
L_NG    = 4.8                          # tCO₂/MWh
RU_MG   = 2.0                          # MW/h for all MTs
RD_MG   = 2.0
ETA_MG_VALS = (0.30, 0.35)             # eff. for each of the two MTs

# --- MT min / max per MG (two MT columns) ----------------------
P_MG_MIN = [
    [2,   2], [2,   2], [2,   2], [1.8, 1.8], [1.8, 1.8], [2,   2],   # MG-0…5
    [2.5, 2.5], [2,   2], [2.2, 2.2], [2, 2], [2.2, 2.2], [2, 2],
    [2,   2], [2,   2],                                                # MG-6…13
    [0,   0], [0,   0], [0,   0], [0,   0], [0,   0], [0,   0], [0, 0] # MG-14…20
]
P_MG_MAX = [
    [6,   6], [6,   6], [6,   6], [5.5, 5.5], [5.5, 5.5], [6,   6],   # MG-0…5
    [8,   8], [7,   7], [7.5, 7.5], [7, 7], [7.5, 7.5], [7, 7],
    [7,   7], [7,   7],                                                # MG-6…13
    [0,   0], [0,   0], [0,   0], [0,   0], [0,   0], [0,   0], [0, 0] # MG-14…20
]

ETA_MG = [list(ETA_MG_VALS) if any(P_MG_MAX[i]) else [0, 0]
          for i in range(NUM_MG)]


# ---------------------------------------------------------------
# 4. CONVERSION & DEVICE PARAMETERS  —  explicit per unit
# ---------------------------------------------------------------
# === PV parameters – indices 0-5 (buses 4,5,7,8,9,11) ==========
PV_PARAMS_MG = {
    0: dict(G_thr=55, G_std=1000, P_rated=3.2),   # MG-0  → bus-4
    1: dict(G_thr=50, G_std=1000, P_rated=3.0),   # MG-1  → bus-5
    2: dict(G_thr=48, G_std=1000, P_rated=2.8),   # MG-2  → bus-7
    3: dict(G_thr=45, G_std=1000, P_rated=2.6),   # MG-3  → bus-8
    4: dict(G_thr=43, G_std=1000, P_rated=2.4),   # MG-4  → bus-9
    5: dict(G_thr=40, G_std=1000, P_rated=2.2),   # MG-5  → bus-11
}

# === Wind-turbine parameters – 9 controller sites = indices 0-8 ==
# (buses 1,2,3,6,13,22,23,27,28 in exactly that order)
WIND_PARAMS_MGC = {
    0: dict(v_ci=3.0, v_r=11.0, v_co=25.0, P_rated=10),  # MGC-0 → bus-1
    1: dict(v_ci=3.2, v_r=11.5, v_co=25.0, P_rated=11),  # MGC-1 → bus-2
    2: dict(v_ci=3.4, v_r=12.0, v_co=25.0, P_rated= 9),  # MGC-2 → bus-3
    3: dict(v_ci=3.1, v_r=11.2, v_co=25.0, P_rated=12),  # MGC-3 → bus-6
    4: dict(v_ci=3.3, v_r=12.5, v_co=25.0, P_rated=10),  # MGC-4 → bus-13
    5: dict(v_ci=3.0, v_r=11.0, v_co=25.0, P_rated=11),  # MGC-5 → bus-22
    6: dict(v_ci=3.2, v_r=12.0, v_co=25.0, P_rated=10),  # MGC-6 → bus-23
    7: dict(v_ci=3.1, v_r=11.5, v_co=25.0, P_rated= 9),  # MGC-7 → bus-27
    8: dict(v_ci=3.4, v_r=12.5, v_co=25.0, P_rated=12),  # MGC-8 → bus-28
}

# === Wind-turbine parameters – MG sites with wind = indices 6-20 ==
# (buses 10,12,14,15,16,17,18,19,20,21,24,25,26,29,30)
WIND_PARAMS_MG = {
     6: dict(v_ci=2.0, v_r=8.0,  v_co=11.0, P_rated=5),   # MG-6  → bus-10
     7: dict(v_ci=2.1, v_r=8.2,  v_co=11.0, P_rated=6),   # MG-7  → bus-12
     8: dict(v_ci=2.2, v_r=8.5,  v_co=11.0, P_rated=5.5), # MG-8  → bus-14
     9: dict(v_ci=2.0, v_r=7.8,  v_co=11.0, P_rated=5),   # MG-9  → bus-15
    10: dict(v_ci=2.3, v_r=8.3,  v_co=11.0, P_rated=6),   # MG-10 → bus-16
    11: dict(v_ci=2.2, v_r=8.1,  v_co=11.0, P_rated=5.5), # MG-11 → bus-17
    12: dict(v_ci=2.1, v_r=8.0,  v_co=11.0, P_rated=6),   # MG-12 → bus-18
    13: dict(v_ci=2.4, v_r=8.4,  v_co=11.0, P_rated=6.5), # MG-13 → bus-19
    14: dict(v_ci=2.0, v_r=7.5,  v_co=11.0, P_rated=6),   # MG-14 → bus-20
    15: dict(v_ci=2.2, v_r=7.8,  v_co=11.0, P_rated=6.5), # MG-15 → bus-21
    16: dict(v_ci=2.1, v_r=7.6,  v_co=11.0, P_rated=6),   # MG-16 → bus-24
    17: dict(v_ci=2.3, v_r=7.9,  v_co=11.0, P_rated=6.5), # MG-17 → bus-25
    18: dict(v_ci=2.0, v_r=7.5,  v_co=11.0, P_rated=6),   # MG-18 → bus-26
    19: dict(v_ci=2.2, v_r=7.8,  v_co=11.0, P_rated=6.5), # MG-19 → bus-29
    20: dict(v_ci=2.1, v_r=7.6,  v_co=11.0, P_rated=6),   # MG-20 → bus-30
}

# Curtailment penalty (controllers + 15 MG turbines)
C_WIND = [0.0005] * (len(WIND_PARAMS_MGC) + len(WIND_PARAMS_MG))

# ---------------------------------------------------------------
# 5.3  Renewable caps (useful for reporting only)
# ---------------------------------------------------------------
RENEW_CAP = {
    **{k: WIND_PARAMS_MG[k]['P_rated'] for k in WIND_PARAMS_MG},  # MG wind
    **{k: PV_PARAMS_MG[k]['P_rated']   for k in PV_PARAMS_MG},    # MG PV
}

# ---------------------------------------------------------------
# 5.4  Battery parameters – MG indices 0-5 (same six PV MGs)
# ---------------------------------------------------------------
BATT_MG   = list(range(0, 6))           # indices of MGs with BESS
E_R       = [0.30, 0.32, 0.28, 0.26, 0.25, 0.27]   # MWh
E_init    = [0.07, 0.08, 0.06, 0.06, 0.06, 0.07]
E_end     = [0.07, 0.08, 0.06, 0.06, 0.06, 0.07]
P_chr_max = [0.06, 0.06, 0.05, 0.05, 0.05, 0.05]
P_dis_max = [0.06, 0.06, 0.05, 0.05, 0.05, 0.05]
SOC_min   = [0.2] * 6
SOC_max   = [0.8] * 6
ETA_CHR   = [0.92] * 6
ETA_DIS   = [0.92] * 6
K_CHR, K_DIS = 0.002, 0.001


# ---------------------------------------------------------------
# 5.6  Transmission tie-lines – **GSF/PTDF placeholders**
# ---------------------------------------------------------------
# ➜  Replace with real 30-bus PTDF vectors (length = 9 + 21 = 30)
LINES = {
    'L12'   : {'limit': 20, 'gsf': [ 0.637187, -0.20191, 0.12766, 0.020392, -0.116623,	-0.031336,	-0.065451,	-0.031213,	-0.022587,	-0.018004,	-0.022587,	-0.000638,	-0.000638,	-0.003155,	-0.005091,	-0.008028,	-0.015048,	-0.0096,	-0.012265,	-0.0137,	-0.017653,	-0.017552,	-0.009779,	-0.016108,	-0.021194,	-0.021194,	-0.024431,	-0.030596,	-0.024431,	-0.024431]},
    'L13'   : {'limit': 50, 'gsf': [ 0.329479,	0.168576,	-0.160994,	-0.053725,	0.083289,	-0.001998,	0.032117,	-0.002121,	-0.010747,	-0.01533,	-0.010747,	-0.032695,	-0.032695,	-0.030178,	-0.028242,	-0.025306,	-0.018285,	-0.023733,	-0.021068,	-0.019634,	-0.015681,	-0.015781,	-0.023554,	-0.017226,	-0.012139,	-0.012139,	-0.008903,	-0.002737,	-0.008903,	-0.008903]},
    'L24'   : {'limit': 20, 'gsf': [ 0.213034,	0.291493,	-0.035421,	-0.087727,	0.146003,	0.000514,	0.05871,	0.000304,	-0.014411,	-0.022229,	-0.014411,	-0.051853,	-0.051853,	-0.047559,	-0.044256,	-0.039247,	-0.027271,	-0.036564,	-0.032019,	-0.029571,	-0.022828,	-0.022999,	-0.036259,	-0.025463,	-0.016787,	-0.016787,	-0.011265,	-0.000748,	-0.011265,	-0.011265]},
    'L34'   : {'limit': 20, 'gsf': [ 0.296146,	0.135243,	0.805673,	-0.087058,	0.049956,	-0.035331,	-0.001216,	-0.035454,	-0.04408,	-0.048663,	-0.04408,	-0.066029,	-0.066029,	-0.063512,	-0.061575,	-0.058639,	-0.051619,	-0.057066,	-0.054402,	-0.052967,	-0.049014,	-0.049114,	-0.056888,	-0.050559,	-0.045473,	-0.045473,	-0.042236,	-0.036071,	-0.042236,	-0.042236]},
    'L25'   : {'limit': 20, 'gsf': [ 0.137381,	0.162967,	0.056358,	0.039301,	-0.420585,	-0.004137,	-0.170716,	-0.004034,	0.00321,	0.007058,	0.00321,	0.021641,	0.021641,	0.019528,	0.017902,	0.015436,	0.009541,	0.014115,	0.011878,	0.010673,	0.007353,	0.007438,	0.013965,	0.008651,	0.004379,	0.004379,	0.001661,	-0.003516,	0.001661,	0.001661]},
    'L26'   : {'limit':  20, 'gsf': [ 0.253439,	0.310297,	0.073389,	0.035484,	0.124626,	-0.061046,	0.013223,	-0.060816,	-0.044719,	-0.036167,	-0.044719,	-0.00376,	-0.00376,	-0.008457,	-0.01207,	-0.01755,	-0.030651,	-0.020485,	-0.025457,	-0.028134,	-0.035511,	-0.035324,	-0.020818,	-0.032628,	-0.04212,	-0.04212,	-0.04816,	-0.059665,	-0.04816,	-0.04816]},
    'L46'   : {'limit': 20, 'gsf': [0.235083,	0.157492,	0.480788,	0.532516,	-0.059699,	-0.276891,	-0.190015,	-0.274962,	-0.139988,	-0.068277,	-0.139988,	0.203453,	0.203453,	0.164068,	0.133773,	0.087823,	-0.022025,	0.063216,	0.021523,	-0.000927,	-0.062782,	-0.061212,	0.06042,	-0.038607,	-0.118196,	-0.118196,	-0.168843,	-0.265315,	-0.168843,	-0.168843]},
    'L57'   : {'limit':  20, 'gsf': [ 0.104048,	0.129634,	0.023025,	0.005968,	0.546082,	-0.037471,	-0.20405,	-0.037367,	-0.030123,	-0.026275,	-0.030123,	-0.011692,	-0.011692,	-0.013806,	-0.015432,	-0.017898,	-0.023793,	-0.019218,	-0.021456,	-0.02266,	-0.02598,	-0.025896,	-0.019368,	-0.024683,	-0.028954,	-0.028954,	-0.031672,	-0.036849,	-0.031672,	-0.031672]},
    'L67'   : {'limit':  20, 'gsf': [ -0.070714,	-0.0963,	0.010308,	0.027366,	-0.512748,	0.070804,	-0.762617,	0.0707,	0.063457,	0.059608,	0.063457,	0.045025,	0.045025,	0.047139,	0.048765,	0.051231,	0.057126,	0.052551,	0.054789,	0.055994,	0.059313,	0.059229,	0.052702,	0.058016,	0.062287,	0.062287,	0.065005,	0.070183,	0.065005,	0.065005]},
    'L68'   : {'limit': 20, 'gsf': [ 0.065404,	0.065589,	0.064818,	0.064695,	0.066107,	0.066624,	0.066417,	-0.79879,	0.053269,	0.046274,	0.053269,	0.049985,	0.049985,	0.047196,	0.045051,	0.048406,	0.046906,	0.045478,	0.04573,	0.045866,	0.041641,	0.040318,	0.034926,	0.021258,	-0.030391,	-0.030391,	-0.063259,	-0.125863,	-0.063259,	-0.063259]},
    'L69' : {'limit':  50, 'gsf': [ 0.205603,	0.208103,	0.197687,	0.196021,	0.2151,	0.222098,	0.219299,	0.219554,	-0.394587,	-0.193803,	-0.394587,	-0.00279,	-0.00279,	-0.023313,	-0.0391,	-0.084072,	-0.16129,	-0.093123,	-0.125046,	-0.142236,	-0.174962,	-0.169579,	-0.061636,	-0.092061,	0.012871,	0.012871,	0.079645,	0.206835,	0.079645,	0.079645]},
    'L6_10' : {'limit':  20, 'gsf': [ 0.104392,	0.105821,	0.099869,	0.098917,	0.109819,	0.113818,	0.112218,	0.112364,	-0.042145,	-0.12384,	-0.042145,	-0.014689,	-0.014689,	-0.026417,	-0.035438,	-0.061136,	-0.105261,	-0.066309,	-0.08455,	-0.094373,	-0.113074,	-0.109997,	-0.048316,	-0.065701,	-0.005741,	-0.005741,	0.032416,	0.105096,	0.032416,	0.032416]},
    'L9_11' : {'limit':  20, 'gsf': [ 0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	-0.966667,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333]},
    'L9_10'   : {'limit':  20, 'gsf': [0.138936,	0.141436,	0.131021,	0.129354,	0.148434,	0.155431,	0.152632,	0.152887,	0.538746,	-0.26047,	0.538746,	-0.069456,	-0.069456,	-0.08998,	-0.105767,	-0.150739,	-0.227957,	-0.15979,	-0.191713,	-0.208902,	-0.241629,	-0.236246,	-0.128303,	-0.158727,	-0.053796,	-0.053796,	0.012979,	0.140168,	0.012979,	0.012979]},
    'L4_12'   : {'limit':  50, 'gsf': [ 0.240763,	0.23591,	0.256131,	0.259366,	0.222326,	0.208741,	0.214175,	0.206478,	0.048164,	-0.035949,	0.048164,	-0.354667,	-0.354667,	-0.308472,	-0.272938,	-0.219042,	-0.090199,	-0.19018,	-0.141277,	-0.114945,	-0.042393,	-0.044234,	-0.1869,	-0.070749,	0.022603,	0.022603,	0.082009,	0.195163,	0.082009,	0.082009]},
    'L12_13' : {'limit':  20, 'gsf': [ 0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	-0.966667,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333]},
    'L12_14' : {'limit':  20, 'gsf': [ 0.041722,	0.041141,	0.043561,	0.043949,	0.039515,	0.03789,	0.03854,	0.037461,	0.021313,	0.01263,	0.021313,	0.090144,	0.090144,	-0.449552,	-0.095471,	0.057159,	0.025824,	-0.057721,	-0.035415,	-0.023404,	0.007495,	0.006028,	-0.06127,	-0.015099,	0.0026,	0.0026,	0.013862,	0.035315,	0.013862,	0.013862]},
    'L12_15': {'limit':  20, 'gsf': [ 0.09635,	0.094295,	0.102859,	0.104229,	0.088542,	0.082789,	0.08509,	0.081271,	0.024134,	-0.00659,	0.024134,	0.267688,	0.267688,	-0.103542,	-0.389103,	0.150974,	0.040096,	-0.255527,	-0.176596,	-0.134094,	-0.02476,	-0.029952,	-0.268085,	-0.104709,	-0.042083,	-0.042083,	-0.00223,	0.07368,	-0.00223,	-0.00223]},
    'L12_16': {'limit':  20, 'gsf': [ 0.036024,	0.033807,	0.043044,	0.044522,	0.027602,	0.021396,	0.023878,	0.02108,	-0.06395,	-0.108656,	-0.06395,	0.220834,	0.220834,	0.177955,	0.14497,	-0.493842,	-0.222785,	0.056403,	0.004067,	-0.024114,	-0.091795,	-0.086978,	0.075788,	-0.017608,	-0.00458,	-0.00458,	0.00371,	0.019501,	0.00371,	0.00371]},
    'L14_15': {'limit':  20, 'gsf': [ 0.008389,	0.007808,	0.010228,	0.010615,	0.006182,	0.004556,	0.005207,	0.004127,	-0.01202,	-0.020703,	-0.01202,	0.05681,	0.05681,	0.517115,	-0.128805,	0.023826,	-0.007509,	-0.091055,	-0.068748,	-0.056737,	-0.025838,	-0.027305,	-0.094604,	-0.048432,	-0.030734,	-0.030734,	-0.019471,	0.001982,	-0.019471,	-0.019471]},
    'L16_17': {'limit':  20, 'gsf': [ 0.002691,	0.000474,	0.009711,	0.011189,	-0.005732,	-0.011937,	-0.009455,	-0.012253,	-0.097284,	-0.141989,	-0.097284,	0.187501,	0.187501,	0.144621,	0.111637,	0.472824,	-0.256118,	0.023069,	-0.029266,	-0.057447,	-0.125128,	-0.120311,	0.042455,	-0.050941,	-0.037913,	-0.037913,	-0.029623,	-0.013832,	-0.029623,	-0.029623]},
    'L15_18': {'limit':  50, 'gsf': [ 0.036094,	0.034864,	0.039988,	0.040808,	0.031422,	0.027979,	0.029356,	0.028057,	-0.023588,	-0.0506,	-0.023588,	0.138613,	0.138613,	0.183226,	0.217544,	0.058097,	-0.018394,	-0.526887,	-0.375869,	-0.294552,	-0.034272,	-0.029607,	0.14096,	0.037571,	0.034367,	0.034367,	0.032329,	0.028445,	0.032329,	0.032329]},
    'L18_19': {'limit':  20, 'gsf': [ 0.00276,	0.001531,	0.006655,	0.007475,	-0.001912,	-0.005354,	-0.003977,	-0.005276,	-0.056922,	-0.083934,	-0.056922,	0.10528,	0.10528,	0.149893,	0.184211,	0.024763,	-0.051727,	0.43978,	-0.409203,	-0.327885,	-0.067605,	-0.06294,	0.107627,	0.004238,	0.001034,	0.001034,	-0.001005,	-0.004888,	-0.001005,	-0.001005]},
    'L19_20': {'limit':  20, 'gsf': [ -0.030573,	-0.031803,	-0.026679,	-0.025859,	-0.035245,	-0.038687,	-0.03731,	-0.03861,	-0.090255,	-0.117267,	-0.090255,	0.071946,	0.071946,	0.11656,	0.150878,	-0.00857,	-0.08506,	0.406446,	0.557464,	-0.361219,	-0.100939,	-0.096274,	0.074293,	-0.029095,	-0.032299,	-0.032299,	-0.034338,	-0.038221,	-0.034338,	-0.034338]},
    'L10_20': {'limit':  20, 'gsf': [ 0.063906,	0.065136,	0.060012,	0.059192,	0.068578,	0.072021,	0.070644,	0.071943,	0.123588,	0.1506,	0.123588,	-0.038613,	-0.038613,	-0.083226,	-0.117544,	0.041903,	0.118394,	-0.373113,	-0.524131,	-0.605448,	0.134272,	0.129607,	-0.04096,	0.062429,	0.065633,	0.065633,	0.067671,	0.071555,	0.067671,	0.067671]},
    'L10_17': {'limit':  20, 'gsf': [ 0.030643,	0.03286,	0.023623,	0.022145,	0.039065,	0.045271,	0.042788,	0.045586,	0.130617,	0.175322,	0.130617,	-0.154168,	-0.154168,	-0.111288,	-0.078304,	-0.439491,	-0.710548,	0.010264,	0.0626,	0.09078,	0.158462,	0.153644,	-0.009122,	0.084274,	0.071247,	0.071247,	0.062956,	0.047165,	0.062956,	0.062956]},
    'L10_21': {'limit':  20, 'gsf': [ 0.074932,	0.075233,	0.073979,	0.073778,	0.076075,	0.076918,	0.076581,	0.074271,	0.133441,	0.163049,	0.133441,	0.049841,	0.049841,	0.030768,	0.016097,	0.098015,	0.14378,	0.067413,	0.097737,	0.114065,	-0.506036,	-0.411489,	-0.097141,	-0.250013,	-0.140815,	-0.140815,	-0.071326,	0.061035,	-0.071326,	-0.071326]},
    'L10_22': {'limit':  20, 'gsf': [ 0.040515,	0.040695,	0.039943,	0.039822,	0.041201,	0.041706,	0.041504,	0.040118,	0.07562,	0.093385,	0.07562,	0.02546,	0.02546,	0.014016,	0.005214,	0.054364,	0.081823,	0.036004,	0.054198,	0.063995,	-0.174733,	-0.251338,	-0.062729,	-0.154452,	-0.088934,	-0.088934,	-0.04724,	0.032176,	-0.04724,	-0.04724]},
    'L21_22': {'limit':  20, 'gsf': [ 0.041599,	0.041899,	0.040645,	0.040445,	0.042742,	0.043584,	0.043247,	0.040937,	0.100108,	0.129716,	0.100108,	0.016508,	0.016508,	-0.002565,	-0.017237,	0.064681,	0.110446,	0.03408,	0.064404,	0.080732,	0.46063,	-0.444823,	-0.130475,	-0.283346,	-0.174149,	-0.174149,	-0.104659,	0.027701,	-0.104659,	-0.104659]},
    'L15_23': {'limit':  20, 'gsf': [ 0.035312,	0.033906,	0.039765,	0.040703,	0.029969,	0.026032,	0.027607,	0.024007,	0.002369,	-0.010026,	0.002369,	0.152552,	0.152552,	0.197013,	0.231214,	0.08337,	0.017647,	0.146972,	0.097192,	0.070387,	-0.049659,	-0.060983,	-0.536982,	-0.224046,	-0.140518,	-0.140518,	-0.087363,	0.013883,	-0.087363,	-0.087363]},
    'L22_24': {'limit':  20, 'gsf': [ 0.04878,	0.049261,	0.047255,	0.046934,	0.050609,	0.051957,	0.051418,	0.047722,	0.142395,	0.189767,	0.142395,	0.008635,	0.008635,	-0.021882,	-0.045357,	0.085713,	0.158936,	0.03675,	0.085268,	0.111393,	0.252564,	0.270506,	-0.226538,	-0.471132,	-0.296416,	-0.296416,	-0.185233,	0.026544,	-0.185233,	-0.185233]},
    'L23_24': {'limit':  20, 'gsf': [ 0.001979,	0.000572,	0.006432,	0.00737,	-0.003364,	-0.007301,	-0.005726,	-0.009326,	-0.030964,	-0.043359,	-0.030964,	0.119219,	0.119219,	0.16368,	0.197881,	0.050037,	-0.015687,	0.113638,	0.063859,	0.037054,	-0.082993,	-0.094317,	0.429685,	-0.257379,	-0.173851,	-0.173851,	-0.120697,	-0.01945,	-0.120697,	-0.120697]},
    'L24_25': {'limit':  20, 'gsf': [ 0.017425,	0.0165,	0.020354,	0.02097,	0.013912,	0.011323,	0.012359,	0.005063,	0.078098,	0.113075,	0.078098,	0.09452,	0.09452,	0.108465,	0.119191,	0.102416,	0.109917,	0.117055,	0.115793,	0.115114,	0.136238,	0.142856,	0.169814,	0.238156,	-0.5036,	-0.5036,	-0.339263,	-0.02624,	-0.339263,	-0.339263]},
    'L25_26': {'limit':  20, 'gsf': [ 0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	-0.966667,	0.033333,	0.033333,	0.033333,	0.033333]},
    'L25_27': {'limit':  20, 'gsf': [ -0.049241,	-0.050166,	-0.046313,	-0.045696,	-0.052755,	-0.055344,	-0.054308,	-0.061604,	0.011431,	0.046408,	0.011431,	0.027854,	0.027854,	0.041798,	0.052524,	0.035749,	0.04325,	0.050389,	0.049126,	0.048447,	0.069571,	0.076189,	0.103148,	0.171489,	0.429733,	0.429733,	-0.40593,	-0.092906,	-0.40593,	-0.40593]},
    'L28_27': {'limit':  50, 'gsf': [ 0.149241,	0.150166,	0.146313,	0.145696,	0.152755,	0.155344,	0.154308,	0.161604,	0.088569,	0.053592,	0.088569,	0.072146,	0.072146,	0.058202,	0.047476,	0.064251,	0.05675,	0.049611,	0.050874,	0.051553,	0.030429,	0.023811,	-0.003148,	-0.071489,	-0.329733,	-0.329733,	-0.49407,	0.192906,	-0.49407,	-0.49407]},
    'L27_29': {'limit':  20, 'gsf': [ 0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	-0.676871,	-0.370748]},
    'L27_30': {'limit':  20, 'gsf': [ 0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	-0.256463,	-0.562585]},
    'L29_30': {'limit':  20, 'gsf': [ 0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.289796,	-0.404082]},
    'L8_28': {'limit':  20, 'gsf': [ 0.032071,	0.032255,	0.031485,	0.031362,	0.032773,	0.033291,	0.033084,	0.167876,	0.019936,	0.012941,	0.019936,	0.016651,	0.016651,	0.013863,	0.011717,	0.015072,	0.013572,	0.012145,	0.012397,	0.012533,	0.008308,	0.006984,	0.001593,	-0.012076,	-0.063724,	-0.063724,	-0.096592,	-0.159196,	-0.096592,	-0.096592]},
    'L6_28': {'limit':  20, 'gsf': [ 0.150504,	0.151244,	0.148161,	0.147668,	0.153315,	0.155386,	0.154558,	0.027061,	0.101966,	0.073985,	0.101966,	0.088828,	0.088828,	0.077673,	0.069092,	0.082512,	0.076511,	0.0708,	0.07181,	0.072354,	0.055454,	0.05016,	0.028593,	-0.02608,	-0.232675,	-0.232675,	-0.364145,	-0.614564,	-0.364145,	-0.364145]},
}


# ---------------------------------------------------------------
# 2. HELPER CONVERSION FUNCTIONS  (unchanged)
# ---------------------------------------------------------------
def solar_power_from_irradiance(G, G_thr, G_std, P_rated):
    """Simple 3-segment PV curve."""
    G = np.asarray(G)
    P = np.zeros_like(G, dtype=float)
    mask1 = (G > G_thr) & (G <= G_std)
    mask2 =  G > G_std
    P[mask1] = P_rated * G[mask1] / G_std
    P[mask2] = P_rated
    return P


def wind_power_from_speed(v, v_ci, v_r, v_co, P_rated):
    """Four-segment wind-turbine curve."""
    v = np.asarray(v)
    P = np.zeros_like(v, dtype=float)
    mask_lin = (v >= v_ci) & (v < v_r)
    mask_rtd = (v >= v_r) & (v <= v_co)
    P[mask_lin] = P_rated * (v[mask_lin]**3 - v_ci**3) / (v_r**3 - v_ci**3)
    P[mask_rtd] = P_rated
    return P

# -----------------------------------------------------------------
# Helper – pretty-printer for one hour (30-bus layout)
# -----------------------------------------------------------------
def _report_hour_30bus(
    t: int,
    *,
    P_grid, P_mt, P_WIND_MGC, u_on,
    G_red, ATC, Flow,
    P_mg, P_RENEW, P_exc,
    P_chr, P_dis, E,
    D_red, D
) -> None:
    """
    Prints a clean snapshot of hour *t* for the 30-bus test case.

    • Controllers are reported in the order WIND_MGCs.
    • Micro-grids MG1…MG21 are reported in ascending MG number.
    """

    print(f"\nHour {t}:")
    # ---------- Controllers -----------------------------------
    for i in range(NUM_MGC):         # 9 controllers
        grid = P_grid[i][t].x or 0.0
        mt   = P_mt[i][t].x   or 0.0
        on   = int(u_on[i][t].x) if u_on[i][t].x is not None else 0
        wind = P_WIND_MGC[i][t]
        print(f"  MGC@Bus{i+1}: Grid={grid:7.2f}, MT={mt:7.2f}, "
              f"Wind={wind:7.2f}, On={on}")

    # ---------- Redispatch & ATC ------------------------------
    red_str = ", ".join(f"G{i+1}={G_red[i][t].x or 0:7.2f}"
                        for i in range(NUM_MGC))
    print(f"  Redispatch: {red_str}, ATC={ATC[t].x or 0:7.2f}")

    # ---------- Selected line flows -----------------
    show_lines = ['L4_12', 'L6_10', 'L9_10','L28_27', 'L10_17', 'L10_20', 'L23_24']
    for name in show_lines:
        print(f"  Flow({name})={Flow[name][t].x or 0.0:8.2f}", end="")
    print()

    # ---------- Micro-grids -----------------------------------
    for mg in range(NUM_MG):                           # MG1 … MG21
        gens   = [P_mg[mg][g][t].x or 0.0 for g in range(2)]
        renew  = P_RENEW[mg][t]
        excs   = [P_exc[i][mg][t].x or 0.0 for i in range(NUM_MGC)]
        batt_net = 0.0
        if mg in BATT_MG:
            batt_net = (P_dis[mg][t].x or 0.0) - (P_chr[mg][t].x or 0.0)
            e_val = E[mg][t].x or 0.0
        total = sum(gens) + renew + batt_net + sum(excs)
        red   = D_red[mg][t].x or 0.0

        label = f"MG{mg:02d}"
        if mg in BATT_MG:
            print(f"  {label}: Dem={D[mg][t]:7.2f}, "
                  f"GTs={np.round(gens,2)}, Ren={renew:7.2f}, "
                  f"Chr={(P_chr[mg][t].x or 0):7.2f}, "
                  f"Dis={(P_dis[mg][t].x or 0):7.2f}, "
                  f"E={e_val:7.2f}, "
                  f"Exc={np.round(excs,2)}, Tot={total:8.2f}, "
                  f"Redisp={red:7.2f}")
        else:
            print(f"  {label}: Dem={D[mg][t]:7.2f}, "
                  f"GTs={np.round(gens,2)}, Ren={renew:7.2f}, "
                  f"Exc={np.round(excs,2)}, Tot={total:8.2f}, "
                  f"Redisp={red:7.2f}")

# ----------------------------------------------------------------
# 3.  SOLVER CORE  – 30-bus MILP
# ----------------------------------------------------------------

def solve_30bus_milp(
    *,
    wind_mgc: Dict[int, Sequence[float]],   # keys = WIND_MGCs
    wind_mg : Dict[int, Sequence[float]],   # keys = WIND_MGs
    irr_pv  : Dict[int, Sequence[float]],   # keys = PV_MGs
    load_D  : np.ndarray,                   # shape (21, 24)   MG1…MG21
    time_limit: int | None = None,
    show_output: bool = False,
) -> Tuple[bool, float | None]:
    """
    Build & solve the single-level MILP for the 30-bus network.
    All time-series arguments must be 24-hour arrays.
    """

    # ------------ (A) Renewable power time-series --------------
    P_WIND_MGC = {
        i: wind_power_from_speed(
            wind_mgc[i],
            WIND_PARAMS_MGC[i]['v_ci'],
            WIND_PARAMS_MGC[i]['v_r'],
            WIND_PARAMS_MGC[i]['v_co'],
            WIND_PARAMS_MGC[i]['P_rated']
        )
        for i in WIND_MGCs
    }

    P_RENEW: Dict[int, np.ndarray] = {}

    for k in range(6, 21):
        p = WIND_PARAMS_MG[k]
        P_RENEW[k] = wind_power_from_speed(
            wind_mg[k], p['v_ci'], p['v_r'], p['v_co'], p['P_rated']
        )

    for k in range(0, 6):
        p = PV_PARAMS_MG[k]
        P_RENEW[k] = solar_power_from_irradiance(
            irr_pv[k], p['G_thr'], p['G_std'], p['P_rated']
        )

    # ------------ (B) BUILD MILP MODEL -----------------------------
    m = Model(sense=MAXIMIZE)
    if time_limit:
        m.max_seconds = time_limit

    # === 7.1-A  Controller-side variables  (9 MGCs) ================
    P_mt    = {i: [m.add_var(lb=0, ub=P_MT_MGC_MAX[i])  for t in range(T)] for i in WIND_MGCs}
    u_on    = {i: [m.add_var(var_type=BINARY)             for t in range(T)] for i in WIND_MGCs}
    v_start = {i: [m.add_var(var_type=BINARY)             for t in range(T)] for i in WIND_MGCs}
    w_stop  = {i: [m.add_var(var_type=BINARY)             for t in range(T)] for i in WIND_MGCs}
    P_grid  = {i: [m.add_var(lb=0, ub=P_GRID_MAX[i])    for t in range(T)] for i in WIND_MGCs}

    # Exports from controller → each micro-grid
    P_exc = {
        i: {mg: [m.add_var(lb=0, ub=P_MGC_MAX) for t in range(T)] for mg in range(NUM_MG)} for i in WIND_MGCs}

    # Redispatch slack, load shed, ATC
    G_red = {i: [m.add_var(lb=0, ub=35) for t in range(T)] for i in WIND_MGCs}
    D_red = {mg:  [m.add_var(lb=0, ub=35) for t in range(T)] for mg  in range(NUM_MG)}
    ATC   = [m.add_var(lb=0) for t in range(T)]

    # Line-flow variables
    Flow = {
        name: [m.add_var(lb=-info['limit'], ub=info['limit']) for t in range(T)]
        for name, info in LINES.items()
    }

    # === 7.1-B  Micro-grid dispatch variables (21 MGs) =============
    P_mg = {
        mg: {g: [m.add_var(lb=P_MG_MIN[mg][g], ub=P_MG_MAX[mg][g]) for t in range(T)]
            for g in range(2)}
        for mg in range(NUM_MG)
    }

    lam = {mg: [m.add_var() for t in range(T)] for mg in range(NUM_MG)}

    alpha_min = {mg: {g: [m.add_var(lb=0) for t in range(T)] for g in range(2)} for mg in range(NUM_MG)}
    alpha_max = {mg: {g: [m.add_var(lb=0) for t in range(T)] for g in range(2)} for mg in range(NUM_MG)}
    beta_up   = {mg: {g: [m.add_var(lb=0) for t in range(T)] for g in range(2)} for mg in range(NUM_MG)}
    beta_dn   = {mg: {g: [m.add_var(lb=0) for t in range(T)] for g in range(2)} for mg in range(NUM_MG)}

    mu_min = {mg: {i: [m.add_var(lb=0) for t in range(T)] for i in WIND_MGCs} for mg in range(NUM_MG)}
    mu_max = {mg: {i: [m.add_var(lb=0) for t in range(T)] for i in WIND_MGCs} for mg in range(NUM_MG)}

    z_min = {mg: {g: [m.add_var(var_type=BINARY) for t in range(T)] for g in range(2)} for mg in range(NUM_MG)}
    z_max = {mg: {g: [m.add_var(var_type=BINARY) for t in range(T)] for g in range(2)} for mg in range(NUM_MG)}
    z_up  = {mg: {g: [m.add_var(var_type=BINARY) for t in range(T)] for g in range(2)} for mg in range(NUM_MG)}
    z_dn  = {mg: {g: [m.add_var(var_type=BINARY) for t in range(T)] for g in range(2)} for mg in range(NUM_MG)}
    z_mu_min = {mg: {i: [m.add_var(var_type=BINARY) for t in range(T)] for i in WIND_MGCs} for mg in range(NUM_MG)}
    z_mu_max = {mg: {i: [m.add_var(var_type=BINARY) for t in range(T)] for i in WIND_MGCs} for mg in range(NUM_MG)}

    # === 7.1-C  Battery variables (only MGs in BATT_MG) ============
    # ------------------------------------------------------------------
    # 7.1-C  Battery variables  (only MGs in BATT_MG)
    # ------------------------------------------------------------------
    E, P_chr, P_dis = {}, {}, {}

    alpha_soc_min, alpha_soc_max = {}, {}
    z_soc_min,     z_soc_max     = {}, {}

    beta_chr_min,  beta_chr_max  = {}, {}
    z_chr_min,     z_chr_max     = {}, {}

    beta_dis_min,  beta_dis_max  = {}, {}
    z_dis_min,     z_dis_max     = {}, {}

    phi, gamma_end, z_end = {}, {}, {}          # each gets its own dict

    for mg in BATT_MG:
        b_idx = BATT_MG.index(mg)
        E[mg]      = [m.add_var(lb=SOC_min[b_idx]*E_R[b_idx],
                                ub=SOC_max[b_idx]*E_R[b_idx]) for t in range(T)]
        P_chr[mg]  = [m.add_var(lb=0, ub=P_chr_max[b_idx]) for t in range(T)]
        P_dis[mg]  = [m.add_var(lb=0, ub=P_dis_max[b_idx]) for t in range(T)]

        alpha_soc_min[mg] = [m.add_var(lb=0)                 for t in range(T)]
        alpha_soc_max[mg] = [m.add_var(lb=0)                 for t in range(T)]
        z_soc_min[mg]     = [m.add_var(var_type=BINARY)      for t in range(T)]
        z_soc_max[mg]     = [m.add_var(var_type=BINARY)      for t in range(T)]

        beta_chr_min[mg]  = [m.add_var(lb=0)                 for t in range(T)]
        beta_chr_max[mg]  = [m.add_var(lb=0)                 for t in range(T)]
        z_chr_min[mg]     = [m.add_var(var_type=BINARY)      for t in range(T)]
        z_chr_max[mg]     = [m.add_var(var_type=BINARY)      for t in range(T)]

        beta_dis_min[mg]  = [m.add_var(lb=0)                 for t in range(T)]
        beta_dis_max[mg]  = [m.add_var(lb=0)                 for t in range(T)]
        z_dis_min[mg]     = [m.add_var(var_type=BINARY)      for t in range(T)]
        z_dis_max[mg]     = [m.add_var(var_type=BINARY)      for t in range(T)]

        gamma_end[mg] = m.add_var(lb=0)
        z_end[mg]     = m.add_var(var_type=BINARY)
        phi[mg]       = [m.add_var() for t in range(T)]

    # ---------------- 7.2 OBJECTIVE FUNCTION ------------------------
    m.objective = xsum(
        # ➤ Revenue from controller exports to MGs
        xsum(C_EXC[i][t] * P_exc[i][mg][t]
            for i in WIND_MGCs for mg in range(NUM_MG))

        # ➤ Redispatch bonus / ATC revenue
        + LAMBDA_ATC * ATC[t]

        # ➤ Costs (subtract inside maximisation form)
        - xsum(
            (C_NG_MGC[i] / (L_NG_MGC[i] * ETA_MGC[i])) * P_mt[i][t]
            + STARTUP_COST[i]  * v_start[i][t]
            + SHUTDOWN_COST[i] * w_stop[i][t]
            + C_WIND[i]        * P_WIND_MGC[i][t]     # wind curtailment
            + C_GRID[t]          * P_grid[i][t]         # grid imports
            for i in WIND_MGCs
        )
        for t in range(T)
    )

    # --------------------------------------------------
    # 7.3  CONSTRAINTS – UPPER LEVEL  (30-bus version)
    # --------------------------------------------------

    # ---------- 7.3.1  MGC power balance ---------------------------
    for i in WIND_MGCs:
        for t in range(T):
            m += (
                P_mt[i][t] + P_WIND_MGC[i][t] + P_grid[i][t]
                == xsum(P_exc[i][mg][t] for mg in range(NUM_MG))
            )

    # ---------- 7.3.2  Commitment, ramps, min-up / min-down --------
    for i in WIND_MGCs:

        # output limits tied to commitment flag
        for t in range(T):
            m += P_mt[i][t] <= P_MT_MGC_MAX[i] * u_on[i][t]
            m += P_mt[i][t] >= P_MT_MGC_MIN[i] * u_on[i][t]

            # start-up / shut-down logic
            if t == 0:
                m += v_start[i][t] - w_stop[i][t] == u_on[i][t]
                m += w_stop[i][t] == 0
            else:
                m += v_start[i][t] - w_stop[i][t] == u_on[i][t] - u_on[i][t-1]

        # ramp limits
        for t in range(1, T):
            m +=  P_mt[i][t]   - P_mt[i][t-1] <= RU_MGC[i]
            m +=  P_mt[i][t-1] - P_mt[i][t]   <= RD_MGC[i]

        # minimum up-time
        for t in range(T - MIN_UP[i]):
            m += xsum(u_on[i][k] for k in range(t+1, t+1+MIN_UP[i])) \
                >= MIN_UP[i] * v_start[i][t+1]

        # minimum down-time
        for t in range(T - MIN_DOWN[i]):
            m += xsum(1 - u_on[i][k] for k in range(t+1, t+1+MIN_DOWN[i])) \
                >= MIN_DOWN[i] * w_stop[i][t+1]

    # ---------- 7.3.3  Exchange capacity (each MG) -----------------
    for mg in range(NUM_MG):
        for t in range(T):
            m += xsum(P_exc[i][mg][t] for i in WIND_MGCs) <= P_MGC_MAX

    # ---------- 7.3.4  Redispatch balance, ATC & line flows --------
    for t in range(T):

        # --- system-wide redispatch balance ------------------------
        expr_red = xsum(
            G_red[i][t] - (P_mt[i][t] + P_grid[i][t] + P_WIND_MGC[i][t])
            for i in WIND_MGCs
        )

        m += expr_red == xsum(
            D_red[mg][t] - xsum(P_exc[i][mg][t] for i in WIND_MGCs)
            for mg in range(NUM_MG)
        )
        m += ATC[t] == expr_red                    # ATC equals net redispatch

        # --- non-negativity of slack variables --------------------
        for i in WIND_MGCs:
            m += G_red[i][t] >= P_mt[i][t] + P_grid[i][t] + P_WIND_MGC[i][t]
        for mg in range(NUM_MG):
            m += D_red[mg][t] >= xsum(P_exc[i][mg][t] for i in WIND_MGCs)

        # --- line flow definitions via PTDF / GSF -----------------
        # --- line flow definitions via PTDF / GSF -----------------
        for ln, info in LINES.items():
            gsf = info["gsf"]      # length-30 PTDF vector (buses 1…30)

            # -------- explicit flow expression (30 buses) ----------
            flow_expr = (
                gsf[0]  * G_red[0][t]   +  # Bus  1  (MGC-0)
                gsf[1]  * G_red[1][t]   +  # Bus  2  (MGC-1)
                gsf[2]  * G_red[2][t]   +  # Bus  3  (MGC-2)
                gsf[3]  * D_red[0][t]   +  # Bus  4  (MG-0)
                gsf[4]  * D_red[1][t]   +  # Bus  5  (MG-1)
                gsf[5]  * G_red[3][t]   +  # Bus  6  (MGC-3)
                gsf[6]  * D_red[2][t]   +  # Bus  7  (MG-2)
                gsf[7]  * D_red[3][t]   +  # Bus  8  (MG-3)
                gsf[8]  * D_red[4][t]   +  # Bus  9  (MG-4)
                gsf[9]  * D_red[14][t]  +  # Bus 10  (MG-14)
                gsf[10] * D_red[5][t]   +  # Bus 11  (MG-5)
                gsf[11] * D_red[6][t]   +  # Bus 12  (MG-6)
                gsf[12] * G_red[5][t]   +  # Bus 13  (MGC-5)
                gsf[13] * D_red[7][t]   +  # Bus 14  (MG-7)
                gsf[14] * D_red[8][t]   +  # Bus 15  (MG-8)
                gsf[15] * D_red[9][t]   +  # Bus 16  (MG-9)
                gsf[16] * D_red[10][t]  +  # Bus 17  (MG-10)
                gsf[17] * D_red[11][t]  +  # Bus 18  (MG-11)
                gsf[18] * D_red[12][t]  +  # Bus 19  (MG-12)
                gsf[19] * D_red[13][t]  +  # Bus 20  (MG-13)
                gsf[20] * D_red[15][t]  +  # Bus 21  (MG-15)
                gsf[21] * G_red[7][t]   +  # Bus 22  (MGC-7)
                gsf[22] * G_red[6][t]   +  # Bus 23  (MGC-6)
                gsf[23] * D_red[16][t]  +  # Bus 24  (MG-16)
                gsf[24] * D_red[17][t]  +  # Bus 25  (MG-17)
                gsf[25] * D_red[18][t]  +  # Bus 26  (MG-18)
                gsf[26] * G_red[8][t]   +  # Bus 27  (MGC-8)
                gsf[27] * G_red[4][t]   +  # Bus 28  (MGC-4)
                gsf[28] * D_red[19][t]  +  # Bus 29  (MG-19)
                gsf[29] * D_red[20][t]     # Bus 30  (MG-20)
            )

            m += Flow[ln][t] == flow_expr

    # --------------------------------------------------
    # 7.3.5  Micro-grid KKT conditions  (30-bus version)
    # --------------------------------------------------
    for mg in range(NUM_MG):                       # MG1 … MG21

        has_batt = mg in BATT_MG
        b_idx    = BATT_MG.index(mg) if has_batt else None   # 0-based into batt arrays

        for t in range(T):

            # ---------- (a) Power balance -------------------------
            supply_mt  = xsum(P_mg[mg][g][t] for g in range(2))  # 0 if no MT
            supply_ren = P_RENEW[mg][t]                          # wind or PV
            supply_exc = xsum(P_exc[i][mg][t] for i in WIND_MGCs)

            if has_batt:
                m += (supply_mt + supply_ren + P_dis[mg][t]
                    - P_chr[mg][t] + supply_exc == load_D[mg][t])
            else:
                m += (supply_mt + supply_ren + supply_exc == load_D[mg][t])

            # ---------- (b) MT bounds, ramps, complementarity ----
            for g in range(2):

                # Fixed bounds
                m += P_mg[mg][g][t] >= P_MG_MIN[mg][g]
                m += P_mg[mg][g][t] <= P_MG_MAX[mg][g]

                # Ramps
                if t > 0:
                    m +=  P_mg[mg][g][t]   - P_mg[mg][g][t-1] <= RU_MG
                    m +=  P_mg[mg][g][t-1] - P_mg[mg][g][t]   <= RD_MG

                # Min / max output complementarity
                m += alpha_min[mg][g][t] <= M * z_min[mg][g][t]
                m += (P_MG_MIN[mg][g] - P_mg[mg][g][t]) <= M * (1 - z_min[mg][g][t])

                m += alpha_max[mg][g][t] <= M * z_max[mg][g][t]
                m += (P_mg[mg][g][t] - P_MG_MAX[mg][g]) <= M * (1 - z_max[mg][g][t])

                # Ramp complementarity
                if t > 0:
                    m += beta_up[mg][g][t] <= M * z_up[mg][g][t]
                    m += (P_mg[mg][g][t] - P_mg[mg][g][t-1] - RU_MG) \
                        <= M * (1 - z_up[mg][g][t])

                    m += beta_dn[mg][g][t] <= M * z_dn[mg][g][t]
                    m += (P_mg[mg][g][t-1] - P_mg[mg][g][t] - RD_MG) \
                        <= M * (1 - z_dn[mg][g][t])

            # ---------- (c) Exchange dual complementarity ---------
            for i in WIND_MGCs:
                m += mu_min[mg][i][t] <= M * z_mu_min[mg][i][t]
                m += (-P_exc[i][mg][t]) <= M * (1 - z_mu_min[mg][i][t])

                m += mu_max[mg][i][t] <= M * z_mu_max[mg][i][t]
                m += (P_exc[i][mg][t] - P_MGC_MAX) \
                    <= M * (1 - z_mu_max[mg][i][t])

            # ---------- (d) Stationarity – generator outputs ------
            for g in range(2):

                # Δβ term for piece-wise ramps
                if T == 1:
                    delta = 0
                elif t == 0:
                    delta = -beta_up[mg][g][t+1] + beta_dn[mg][g][t+1]
                elif t == T-1:
                    delta =  beta_up[mg][g][t]   - beta_dn[mg][g][t]
                else:
                    delta = ( beta_up[mg][g][t]   - beta_dn[mg][g][t]
                            - beta_up[mg][g][t+1] + beta_dn[mg][g][t+1])

                cost_term = 0 if ETA_MG[mg][g] == 0 else C_NG / (L_NG * ETA_MG[mg][g])

                m += (cost_term - lam[mg][t] - alpha_min[mg][g][t]
                    + alpha_max[mg][g][t] + delta == 0)

            # ---------- (e) Stationarity – exchange power ----------
            for i in WIND_MGCs:
                m += (C_EXC[i][t] - lam[mg][t]
                    - mu_min[mg][i][t] + mu_max[mg][i][t] == 0)

        # ----------------------------------------------------------
        # 7.3.6  Battery constraints (only MGs in BATT_MG)
        # ----------------------------------------------------------
        if has_batt:

            # Initial & final SOC
            m += E[mg][0]    == E_init[b_idx]
            m += E[mg][T-1] >= E_end[b_idx]

            m += gamma_end[mg] <= M * z_end[mg]
            m += (E[mg][T-1] - E_end[b_idx]) <= M * (1 - z_end[mg])

            # Energy dynamics
            for t in range(T-1):
                m += (E[mg][t+1] == E[mg][t]
                    + ETA_CHR[b_idx] * P_chr[mg][t] * Delta_t
                    - P_dis[mg][t] * Delta_t / ETA_DIS[b_idx])

            # Stationarity on SOC chain
            for t in range(T-1):
                m += (phi[mg][t] - phi[mg][t+1]
                    + alpha_soc_max[mg][t] - alpha_soc_min[mg][t] == 0)

            m += (phi[mg][T-1] + gamma_end[mg]
                + alpha_soc_max[mg][T-1] - alpha_soc_min[mg][T-1]
                - 45 == 0)     # retain original constant

            # SOC & power complementarity
            for t in range(T):

                # SOC bounds
                m += alpha_soc_min[mg][t] <= M * z_soc_min[mg][t]
                m += (E[mg][t] - SOC_min[b_idx] * E_R[b_idx]) \
                    <= M * (1 - z_soc_min[mg][t])

                m += alpha_soc_max[mg][t] <= M * z_soc_max[mg][t]
                m += (SOC_max[b_idx] * E_R[b_idx] - E[mg][t]) \
                    <= M * (1 - z_soc_max[mg][t])

                # Charging power bounds
                m += beta_chr_min[mg][t] <= M * z_chr_min[mg][t]
                m += P_chr[mg][t] <= M * (1 - z_chr_min[mg][t])

                m += beta_chr_max[mg][t] <= M * z_chr_max[mg][t]
                m += (P_chr[mg][t] - P_chr_max[b_idx]) \
                    <= M * (1 - z_chr_max[mg][t])

                # Discharging power bounds
                m += beta_dis_min[mg][t] <= M * z_dis_min[mg][t]
                m += P_dis[mg][t] <= M * (1 - z_dis_min[mg][t])

                m += beta_dis_max[mg][t] <= M * z_dis_max[mg][t]
                m += (P_dis[mg][t] - P_dis_max[b_idx]) \
                    <= M * (1 - z_dis_max[mg][t])

                # Stationarity w.r.t charge / discharge
                phi_next = phi[mg][t+1] if t < T-1 else gamma_end[mg]

                m += (K_CHR * Delta_t
                    + lam[mg][t] * ETA_CHR[b_idx] * Delta_t
                    - phi_next
                    - beta_chr_min[mg][t] + beta_chr_max[mg][t] == 0)

                m += (K_DIS * Delta_t
                    - lam[mg][t] * Delta_t / ETA_DIS[b_idx]
                    + phi_next
                    - beta_dis_min[mg][t] + beta_dis_max[mg][t] == 0)

    # ----------------------------------------------------------------
    #  ▶  Solve & pretty-print
    # ----------------------------------------------------------------
    status = m.optimize()
    if status not in (
            OptimizationStatus.OPTIMAL,
            OptimizationStatus.FEASIBLE,
            OptimizationStatus.UNBOUNDED):
        if show_output:
            print(f"❌ MILP ended with status {status.name}")
        return False, None

    if show_output:
        print(f"\n✓ Objective = {m.objective_value:,.2f}  (status: {status.name})")
        for t in range(T):
            _report_hour_30bus(                         # ← 30-bus printer
                t,
                P_grid=P_grid, P_mt=P_mt,
                P_WIND_MGC=P_WIND_MGC, u_on=u_on,
                G_red=G_red, ATC=ATC, Flow=Flow,
                P_mg=P_mg, P_RENEW=P_RENEW, P_exc=P_exc,
                P_chr=P_chr, P_dis=P_dis, E=E,
                D_red=D_red, D=load_D,
            )

    return True, m.objective_value

# ----------------------------------------------------------------
# 5. RL-FACING ADAPTER  – 30-bus naming scheme
# ----------------------------------------------------------------

# ---- 5.1  Keys the RL loop must provide ------------------------
RL_KEYS: set[str] = {
    # Wind at controllers (9 keys)
    *(f"MGC{bus}_wind_pred"  for bus in MGC_BUS.values()),

    # Wind at MG sites with turbines (15 keys)
    *(f"MG{bus}_wind_pred"   for bus in WIND_MG_BUS.values()),

    # Solar irradiance at PV MGs (6 keys)
    *(f"MG{bus}_solar_pred"  for bus in PV_MG_BUS.values()),

    # Hourly load at **every** MG (21 keys)
    *(f"MG{bus}_load_pred"   for bus in MG_BUS.values()),
}

# ---- 5.2  Adapter that converts the forecast dict --------------
def deterministic_day_ahead(
    forecast: Dict[str, Sequence[float]],
    *, show_output: bool = False,
    time_limit: int | None = None
) -> Tuple[bool, float | None]:

    # ── Sanity-check the incoming dict
    missing = RL_KEYS - forecast.keys()
    if missing:
        raise KeyError(
            f"deterministic_day_ahead – missing keys: {sorted(missing)}"
        )

    # ── Slice forecast → the four arrays the MILP needs
    wind_mgc = {
        i: np.asarray(
            forecast[f"MGC{MGC_BUS[i]}_wind_pred"], dtype=float
        )
        for i in MGC
    }

    wind_mg = {
        k: np.asarray(
            forecast[f"MG{WIND_MG_BUS[k]}_wind_pred"], dtype=float
        )
        for k in WIND_MGs
    }

    irr_pv = {
        k: np.asarray(
            forecast[f"MG{PV_MG_BUS[k]}_solar_pred"], dtype=float
        )
        for k in PV_MGs
    }

    load_D = np.vstack([
        np.asarray(
            forecast[f"MG{MG_BUS[k]}_load_pred"], dtype=float
        )
        for k in range(NUM_MG)
    ])

    # ── Call the solver core
    return solve_30bus_milp(
        wind_mgc=wind_mgc,
        wind_mg=wind_mg,
        irr_pv=irr_pv,
        load_D=load_D,
        time_limit=time_limit,
        show_output=show_output,
    )

In [3]:
# ================================================================
#  Hyper-parameters  – Agent 5 (second-batch load series)
# ================================================================

# ϵ-greedy exploration
MIN_EXPLORATION_PROB     = 1e-14
INITIAL_EXPLORATION_PROB = 1.0
EXPLORATION_DECAY        = 0.995          # ε ← ε · decay  (per episode)

# Target-network soft update
TAU          = 1e-3                       # θ_target ← (1-τ)θ_target + τθ_local

# Learning parameters
LR           = 1e-4                       # Adam learning rate
GAMMA        = 0.80                       # discount factor γ
UPDATE_EVERY = 4                          # agent.learn() call interval

# Replay buffer
BUFFER_SIZE  = int(1e6)
BATCH_SIZE   = 128

# Network architecture
# ── 13-dim state → two FC layers
FC1_SIZE = 128
FC2_SIZE = 128

# Training loop limits
MAX_EPISODES = 4000

# Goal / logging
ENV_SOLVED   = 200                        # moving-average reward target
PRINT_EVERY  = 100                        # print progress every N episodes

In [4]:
# ----------------------------------------------------------------
#  Replace / rerun the DQN class definition (Agent 1 variant)
# ----------------------------------------------------------------
class DQN(nn.Module):
    """
    Agent 1 network

    • Input  : 16-dim state vector

    • Output : Q-values for each discrete action of every series

    """

    def __init__(
        self,
        state_size: int,
        fc1_size: int,
        fc2_size: int,
        series_count: int,
        n_choices: int,
        seed: int = 0,
    ):
        super().__init__()
        torch.manual_seed(seed)

        self.series_count = series_count  
        self.n_choices = n_choices       
        self.out_size = series_count * n_choices

        self.net = nn.Sequential(
            nn.Linear(state_size, fc1_size),
            nn.ReLU(),
            nn.Linear(fc1_size, fc2_size),
            nn.ReLU(),
            nn.Linear(fc2_size, self.out_size),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Returns
        -------
        Q : torch.Tensor
            Shape = (batch, series_count, n_choices)
        """
        out = self.net(x)  # (batch, out_size)
        return out.view(-1, self.series_count, self.n_choices)

In [5]:
# ------------------------------------------------------------------
# Replay memory
# ------------------------------------------------------------------
class ReplayMemory:
    """Fixed-size FIFO buffer that stores experience tuples.

    Each tuple contains
        state       : np.ndarray, shape (16,)   – Agent 1 state vector
        action      : np.ndarray, shape (5,)    – integer indices 0…5
        reward      : float
        next_state  : np.ndarray, shape (16,)
        done        : bool
    """

    def __init__(self, buffer_size, batch_size, seed):
        self.batch_size = batch_size
        self.seed = random.seed(seed)
        self.memory = deque(maxlen=buffer_size)
        self.experience = namedtuple(
            "Experience",
            field_names=["state", "action", "reward", "next_state", "done"],
        )

    def add_experience(self, state, action, reward, next_state, done):
        """Save one (s, a, r, s′, done) transition."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        """Randomly sample a batch of experiences and convert to tensors."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(
            np.vstack([e.state for e in experiences if e is not None])
        ).float().to(device)
        actions = torch.from_numpy(
            np.vstack([e.action for e in experiences if e is not None])
        ).long().to(device)
        rewards = torch.from_numpy(
            np.vstack([e.reward for e in experiences if e is not None])
        ).float().to(device)
        next_states = torch.from_numpy(
            np.vstack([e.next_state for e in experiences if e is not None])
        ).float().to(device)
        dones = torch.from_numpy(
            np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)
        ).float().to(device)

        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.memory)

In [6]:
# ------------------------------------------------------------------
#  Deep-Q agent (fixed target network)  –  Agent 1 variant
# ------------------------------------------------------------------
class DeepQAgent_Fixed:
    """
    Wrapper around a pair of DQN networks + replay buffer.

    """
    def __init__(
        self,
        state_size,
        fc1_size,
        fc2_size,
        series_count,
        n_choices,
        seed=0,
    ):
        # ---------- store meta ----------------------------------
        self.state_size = state_size          
        self.series_count = series_count      
        self.n_choices = n_choices            
        self.seed = random.seed(seed)

        # ---------- networks ------------------------------------
        self.q_network = DQN(
            state_size, fc1_size, fc2_size, series_count, n_choices, seed
        ).to(device)
        self.target_network = DQN(
            state_size, fc1_size, fc2_size, series_count, n_choices, seed
        ).to(device)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=LR)

        # ---------- replay buffer -------------------------------
        self.memory = ReplayMemory(BUFFER_SIZE, BATCH_SIZE, seed)
        self.timestep = 0

    # ============================================================
    #  Step / learn
    # ============================================================
    def step(self, state, action, reward, next_state, done):
        """Save transition and, every UPDATE_EVERY steps, learn."""
        self.memory.add_experience(state, action, reward, next_state, done)
        self.timestep += 1
        if self.timestep % UPDATE_EVERY == 0:
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences)

    def learn(self, experiences):
        # --------------------------------------------------------
        #  Unpack tensors
        # --------------------------------------------------------
        (
            states,       
            actions,      
            rewards,      
            next_states,  
            dones,        
        ) = experiences

        # --------------------------------------------------------
        #  Q(s, a) from local network
        # --------------------------------------------------------
        q_logits = self.q_network(states)           
        action_idx = actions.unsqueeze(-1)          
        q_expected = torch.gather(q_logits, 2, action_idx).squeeze(-1)  

        # --------------------------------------------------------
        #  Q_target(s′, a_max) from target network
        # --------------------------------------------------------
        with torch.no_grad():
            next_logits = self.target_network(next_states)  
            best_next = next_logits.max(dim=2)[0]           

        q_target = rewards + GAMMA * best_next * (1 - dones)

        # --------------------------------------------------------
        #  Loss & optimisation
        # --------------------------------------------------------
        loss = F.mse_loss(q_expected, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Soft update target parameters
        self.update_target_network(self.q_network, self.target_network)

    # ------------------------------------------------------------
    #  Helpers
    # ------------------------------------------------------------
    @staticmethod
    def update_target_network(source_network, target_network):
        """Polyak averaging: θ_target ← (1-τ)θ_target + τθ_local."""
        for src_p, tgt_p in zip(source_network.parameters(), target_network.parameters()):
            tgt_p.data.copy_(TAU * src_p.data + (1.0 - TAU) * tgt_p.data)

    def act(self, state, eps=0.0):
        """
        ϵ-greedy action selection.

        Returns
        -------
        action_vec : np.ndarray, shape (5,), dtype int64
                     each element ∈ {0,…,5}
        """
        # ϵ-explore
        if random.random() < eps:
            return np.array(
                [random.randrange(self.n_choices) for _ in range(self.series_count)],
                dtype=np.int64,
            )

        # Greedy
        state_t = torch.from_numpy(np.asarray(state)).float().unsqueeze(0).to(device)
        self.q_network.eval()
        with torch.no_grad():
            logits = self.q_network(state_t)  
        self.q_network.train()

        action_vec = logits.squeeze(0).cpu().numpy().argmax(axis=1)
        return action_vec.astype(np.int64)

    def checkpoint(self, filename: str):
        """Save the local Q-network weights to disk."""
        torch.save(self.q_network.state_dict(), filename)

##### Test the model

In [7]:
#!/usr/bin/env python3
# test_rl_in_the_loop_30bus.py
# --------------------------------------------------------------
# 24-hour test run with TEN DQN agents (30-bus case)
# --------------------------------------------------------------

import numpy as np
import pandas as pd
import torch

# ----------------------------------------------------------------
# 0 ▸  CONFIGURATION
# ----------------------------------------------------------------
CSV_PATH = "combined_full_data_30_test.csv"          # 24-hour forecast file

# ---- 0.1 ▸ Physical-bus groups ---------------------------------
MGC_WIND_BUSES  = [1, 2, 3, 6, 13, 22, 23, 27, 28]      # 9
MG_WIND_BUSES   = [10, 12, 14, 15, 16,
                   17, 18, 19, 20, 21,
                   24, 25, 26, 29, 30]                  # 15
MG_PV_BUSES_1   = [4, 5, 7]                             # 3
MG_PV_BUSES_2   = [8, 9, 11]                            # 3
MG_LOAD_BUSES   = [4, 5, 7, 8, 9, 11, 10,
                   12, 14, 15, 16, 17, 18, 19,
                   20, 21, 24, 25, 26, 29, 30]          # 21
# sanity checks
assert len(MGC_WIND_BUSES) == 9
assert len(MG_PV_BUSES_1) + len(MG_PV_BUSES_2) == 6
assert len(MG_WIND_BUSES) == 15
assert len(MG_LOAD_BUSES) == 21

# ---- 0.2 ▸ TARGET_ORDER (row order in the matrices) --------------
TARGET_ORDER = (
    [f"MGC{b}_wind"  for b in MGC_WIND_BUSES]  +       # 0 – 8
    [f"MG{b}_wind"   for b in MG_WIND_BUSES]   +       # 9 – 23
    [f"MG{b}_solar"  for b in MG_PV_BUSES_1]   +       # 24 – 26
    [f"MG{b}_solar"  for b in MG_PV_BUSES_2]   +       # 27 – 29
    [f"MG{b}_load"   for b in MG_LOAD_BUSES]           # 30 – 50
)
assert len(TARGET_ORDER) == 51

# ---- 0.3 ▸ Agent → slice mapping (indices in TARGET_ORDER) -------
AGENT_SERIES = {
     1: list(range(0,  5)),          # MGC wind buses 1-5   (1,2,3,6,13)
     2: list(range(5,  9)),          # MGC wind buses 6-9   (22,23,27,28)
     3: list(range(24, 27)),         # PV buses 4,5,7
     4: list(range(27, 30)),         # PV buses 8,9,11
     5: list(range( 9, 14)),         # MG-wind buses 10-16   (first 5)
     6: list(range(14, 19)),         # MG-wind buses 17-21   (next 5)
     7: list(range(19, 24)),         # MG-wind buses 24-30   (last 5)
     8: list(range(30, 37)),         # MG-load buses  4-11   (first 7)
     9: list(range(37, 44)),         # MG-load buses 12-19   (next 7)
    10: list(range(44, 51)),         # MG-load buses 20-30   (last 7)
}

CHECKPOINTS = {
     1: "dqn_agent01_mgcWind1-5_final.pth",
     2: "dqn_agent02_mgcWind6-9_final.pth",
     3: "dqn_agent03_pvBus4-5-7_final.pth",
     4: "dqn_agent04_pvBus8-9-11_final.pth",
     5: "dqn_agent05_windMG7-11_final.pth",
     6: "dqn_agent06_windMG12-16_final.pth",
     7: "dqn_agent07_windMG17-21_final.pth",
     8: "dqn_agent08_loadMG1-7_final.pth",
     9: "dqn_agent09_loadMG8-14_final.pth",
    10: "dqn_agent10_loadMG15-21_final.pth",
}

# ---- 0.4 ▸ Discrete multiplier grids (same as training) ----------
WIND_MGC_GRID  = np.array([-2, -1.5, -1, 1, 1.5, 2])
WIND_MG_GRID   = np.array([-2.0, -1.5, -1.0, 1.0, 1.5, 2.0])
PV_GRID        = np.array([-0.4, -0.2, -0.1, -0.08, -0.05, -0.01,
                           0.01, 0.05, 0.08, 0.10, 0.20, 0.4])
LOAD_GRID      = np.array([-10, -8, -5, 5, 8, 10])

ACTION_LISTS = {
     1: WIND_MGC_GRID,
     2: WIND_MGC_GRID,
     3: PV_GRID,
     4: PV_GRID,
     5: WIND_MG_GRID,
     6: WIND_MG_GRID,
     7: WIND_MG_GRID,
     8: LOAD_GRID,
     9: LOAD_GRID,
    10: LOAD_GRID,
}

HIDDEN_SIZES = {a: (128, 128) for a in AGENT_SERIES}

# ---- 0.5 ▸ Row-index → CSV key for forecast_dict ----------------
SERIES_META = []
for idx, name in enumerate(TARGET_ORDER):
    # convert e.g. "MGC13_wind" → "MGC13_wind_pred"
    SERIES_META.append((idx, f"{name}_pred"))

# ----------------------------------------------------------------
# 1 ▸ LOAD 24-H FORECAST MATRICES
# ----------------------------------------------------------------
df = pd.read_csv(CSV_PATH)
assert len(df) == 24, "CSV must contain exactly 24 rows"

hours = df["Hour"].to_numpy()

pred = np.vstack([df[f"{n}_pred"].to_numpy() for n in TARGET_ORDER])
mean = np.vstack([df[f"{n}_mean"].to_numpy() for n in TARGET_ORDER])
std  = np.vstack([df[f"{n}_std" ].to_numpy() for n in TARGET_ORDER])
# shapes → (51, 24)

In [8]:
# ----------------------------------------------------------------
# 2 ▸  INSTANTIATE & LOAD THE TEN AGENTS
# ----------------------------------------------------------------
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

agents = {}
for aid, rows in AGENT_SERIES.items():
    fc1, fc2 = HIDDEN_SIZES[aid]
    agents[aid] = DeepQAgent_Fixed(
        1 + 3 * len(rows),          # state = Hour + 3×series
        fc1, fc2,
        len(rows),                  # one discrete action per series
        len(ACTION_LISTS[aid]),     # grid length
        seed=0
    )
    agents[aid].q_network.load_state_dict(
        torch.load(CHECKPOINTS[aid], map_location=device)
    )
    agents[aid].target_network.load_state_dict(
        torch.load(CHECKPOINTS[aid], map_location=device)
    )
    agents[aid].q_network.eval()

# ----------------------------------------------------------------
# 3 ▸  ROLL FORWARD HOUR-BY-HOUR  → build hourly “adjusted” values
# ----------------------------------------------------------------
hourly_pred = {h: {} for h in range(24)}       # Hour → { series_key : value }

for h in range(24):

    for aid, rows in AGENT_SERIES.items():

        # ---- build per-agent state vector  [Hour | pred mean std × N]
        state_parts = []
        for r in rows:
            state_parts.extend([pred[r, h], mean[r, h], std[r, h]])
        state_vec = np.asarray([hours[h]] + state_parts, dtype=np.float32)

        # ---- greedy actions  (one per managed series)
        action_idx = agents[aid].act(state_vec, eps=0.0)   # shape (len(rows),)

        # ---- translate index → multiplier → adjusted prediction
        for j, series_idx in enumerate(rows):
            mult = ACTION_LISTS[aid][action_idx[j]]
            adj  = pred[series_idx, h] + mean[series_idx, h] + mult * std[series_idx, h]
            key  = f"{TARGET_ORDER[series_idx]}_pred"      # e.g. “MGC13_wind_pred”
            hourly_pred[h][key] = adj

In [13]:
# ----------------------------------------------------------------
# 4 ▸  PACK INTO 24-H FORECAST DICT & CALL THE MILP SOLVER
# ----------------------------------------------------------------
KEYS_51 = [f"{name}_pred" for name in TARGET_ORDER]   # 51 expected keys

forecast_24h = {
    key: np.asarray([hourly_pred[h][key] for h in range(24)], dtype=float)
    for key in KEYS_51
}

In [ ]:
feasible, objective = deterministic_day_ahead(
    forecast_24h,
    show_output=True,          # set False for quiet run
)

print("\n──────── 24-Hour 10-Agent Test ────────")
print(f"Feasible solution : {feasible}")
print(f"Objective value   : {objective:.2f}")
print("──────────────────────────────────────")